In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jun 25 00:51:49 2025

@author: DELL
"""

import os
import sys
import ast
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings("ignore")
# import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler  # 只保留一个 StandardScaler 导入
import shutil
from tensorflow.keras.models import load_model

#from IPython.display import display
#from tqdm.notebook import tqdm
import tensorflow.keras.backend as K
import shap
import numpy as np
import tensorflow as tf

print("TensorFlow 版本:", tf.__version__)
print("使用的 Keras API:", tf.keras.__version__)
print("shap 版本:", shap.__version__)



# 定义一个函数来解析每行数据
# 定义一个函数来解析每行数据
def parse_line(line):
    parts = line.strip().split('\t')
    crop = parts[0]
    filename = parts[1]
    region = parts[2]
    start_year = parts[3] if parts[3] else None  # 如果为空，则设置为None
    end_year = parts[4] if parts[4] else None    # 如果为空，则设置为None
    pre_name1 = parts[5] 
    pre_name2 = parts[6]
    pre_name3 = parts[7] 
    shp_name = parts[8] 
    flag = parts[9]
    number =  parts[10]
    return crop,filename, region, start_year, end_year,pre_name1,pre_name2,pre_name3,shp_name,flag,number


def extract_selected_variables(inputpath_base):
    inpath_dates = os.path.join(inputpath_base, '01_data','05_buildmodel', '04_selectFeatures','selectFeatures.txt')
    # 构建文件路径
    gs_infornamtion = pd.read_csv(inpath_dates, sep='\t', header=None)
    gs_infornamtion.columns = ['slected_dynamic_features', 'slected_static', 'regionID']
    gs_infornamtion['slected_dynamic_features'] = gs_infornamtion['slected_dynamic_features'].apply(ast.literal_eval)
    gs_infornamtion['slected_static'] = gs_infornamtion['slected_static'].apply(ast.literal_eval)
    return gs_infornamtion
    
modelname ='LSTM';yield_type = 'actual_yield';

data_lines = [
    ###玉米0-13
    
    "01_Maize\t01_China\tII\t2001\t2017\tMaize_ChinaII_\tmaize_8aggChinaII_\tMaize_ChinaII_\tChinagee_up_load_regionII.shp\tB\t2",
    "01_Maize\t01_China\tI\t2001\t2017\tMaize_ChinaI_\tmaize_8aggChinaI_\tMaize_ChinaI_\tChinagee_up_load_region_I.shp\tB\t1",
    "01_Maize\t08_Indonesia\tI\t2001\t2015\tMaize_dailyI_\tMaize_8aggI_\tMaize_I_\tIndonesiagee_up_load.shp\tB\t3",
    "01_Maize\t05_India\tI\t2001\t2018\tMaize_dailyI_\tMaize_8aggI_\tMaize_I_\tIndiagee_up_load.shp\tB\t4",#
    "01_Maize\t07_Ukraine\tI\t2004\t2019\tMaize_dailyI_\tMaize_8aggI_\tMaize_I_\tUkrainegee_up_load.shp\tB\t5", #
    "01_Maize\t03_European\tI\t2001\t2019\tMaize_dailyI_\tMaize_8aggI_\tMaize_I_\tEuropeangee_up_load.shp\tB\t6",#
    "01_Maize\t09_Africa\tI\t2001\t2018\tMaize_dailyI_\tMaize_8aggI_\tMaize_I_\tAfrica5_I.shp\tB\t7",
    "01_Maize\t09_Africa\tII\t2001\t2018\tMaize_dailyII_\tMaize_8aggII_\tMaize_II_\tAfrica5_II.shp\tB\t8",
    "01_Maize\t09_Africa\tIII\t2001\t2018\tMaize_dailyIII_\tMaize_8aggIII_\tMaize_III_\tAfrica5_III.shp\tB\t9",
    "01_Maize\t04_Argentina\tI\t2001\t2017\tMaize_dailyI_\tMaize_8aggI_\tMaize_I_\tArgentinagee_up_load.shp\tB\t10",
    "01_Maize\t03_Brazil\tI\t2001\t2019\tMaize_dailyBrazilI_\tMaize_8aggBrazilI_\thist_BrazilI_\tBrazilgee_up_loadI.shp\tB\t11",
    "01_Maize\t03_Brazil\tII\t2001\t2019\tMaize_dailyBrazilII_\tMaize_8aggBrazilII_\thist_BrazilII_\tBrazilgee_up_loadII.shp\tB\t12",
    "01_Maize\t06_Mexico\tI\t2003\t2018\tMaize_dailyI_\tMaize_8aggSprsum_\tMaize_I_\tMexicogee_up_loadSprsum.shp\tB\t13",
    "01_Maize\t02_US\tI\t2001\t2018\tMaize_USI_\tmaize_8aggUSI_\tMaize_USI_\tUSgee_up_load.shp\tB\t14", 
    
     ###小麦 14-26
    "02_Wheat\t08_Russia\tII\t2001\t2019\tSpringWheat_RussiaII_\tSpringWheat_8aggRussiaII_\tWheat_II_\tRussiagee_up_load_spring.shp\tB\t1",
    "02_Wheat\t08_Russia\tI\t2001\t2019\tWinterWheat_RussiaI_\tWinterWheat_8aggRussiaI_\tWheat_I_\tRussiagee_up_load_winter.shp\tB\t2",
    "02_Wheat\t10_Ukraine\tI\t2004\t2019\tWinterWheat_UkraineI_\tWinterWheat_8aggUkraineI_\tWheat_I_\tUkrainegee_up_load_winter.shp\tB\t3",
    "02_Wheat\t09_European\tI\t2001\t2019\tWinterWheat_European_\tWinterWheat_8aggEuropeanI_\tWheat_I_\tEuropeangee_up_load_winter.shp\tB\t4",
    "02_Wheat\t07_China\tI\t2001\t2017\tSpringWheat_ChinaI_\tSpringWheat_8aggChinaI_\tWheat_I_\tchina_spring_wheat.shp\tB\t5",
    "02_Wheat\t07_China\tII\t2001\t2017\tWinterWheat_ChinaII_\tSpringWheat_8aggChinaI_\tWheat_I_\tChinagee_up_load_winter.shp\tB\t6",
    "02_Wheat\t04_Australia\tI\t2005\t2020\tWinterWheat_AustraliaI_\tWinterWheat_8aggAustaliaI_\tWheat_I_1\tAustraliagee_up_load_winter.shp\tB\t7",
    "02_Wheat\t06_India\tI\t2001\t2018\tWinterWheat_IndiaI_\tWinterWheat_8aggIndiaI_\tWheat_I_\tIndiagee_up_load_winter.shp\tB\t8",
    "02_Wheat\t12_Pakistan\tI\t2001\t2014\tWinterWheat_Pakistan_\tWinterWheat_8aggPakistanI_\tWheat_I_\t'Pakistangee_up_load_winter.shp\tB\t9",
    "02_Wheat\t05_Canada\tI\t2001\t2019\tSpringWheat_Canada_\tSpringWheat_8aggCanada_\tWheat_I_\t'Canadagee_up_load_spring.shp\tA\t10", # 
    "02_Wheat\t02_US\tI\t2001\t2018\tWinterWheat_USII_\tWinterWheat_8aggUSII_\tWheat_II_\tUSgee_up_load_winter.shp\tB\t11",    
    "02_Wheat\t02_US\tII\t2001\t2018\tSpringWheat_USI_\tSpringWheat_8aggUSI_\tWheat_I_\tUSgee_up_load_Spring.shp\tB\t12",
    "02_Wheat\t03_Argentina\tI\t2001\t2017\tWinterWheat_ArgentinaI_\tWinterWheat_8aggArgentinaI_\tWheat_I_\tArgentinagee_up_load_winter.shp\tB\t13",
    
    ###水稻 27
    "03_Rice\t10_Japan\tI\t2001\t2017\tSingleRice_dailyJapan_SingleRice_\tSingleRice_8aggJapan_SingleRice_\tRice_I_1971\tJapangee_up_load_single.shp\tB\t1",
    "03_Rice\t13_Southkorea\tI\t2001\t2018\tSingleRice_dailyKorea_SingleRice_\tSingleRice_8aggKorea_SingleRice_\tRice_I_\tSouthkoreagee_up_load_single.shp\tB\t2",
    "03_Rice\t01_China\tI\t2001\t2017\tSingleRice_dailyChinaI_\tSingleRice_8aggChinaI_\tRice_I_\tChinagee_up_load_rice_I.shp\tB\t3",
    "03_Rice\t01_China\tII\t2001\t2017\tEarlyRice_dailyChinaII_\tEarlyRice_8aggChinaII_\tRice_II_\tChinagee_up_load_rice_II.shp\tB\t4",
    "03_Rice\t08_Philippines\tI\t2001\t2016\tDoubleRice_dailyPhilippines_DoubleRice_\tDoubleRice_8aggPhilippines_DoubleRice_\tRice_I_\thilippinesgee_up_load_double1.shp\tB\t5",
    "03_Rice\t03_Indonesia\tI\t2001\t2015\tSingleRice_dailyIndonesia_SingleRice_\tSingleRice_8aggIndonesia_SingleRice_\tRice_I_\tIndonesiagee_up_load_single.shp\tB\t6", # 
    "03_Rice\t04_Vietnam\tI\t2001\t2018\tDoubleRice_dailyVietnam_DoubleRice_\tDoubleRice_8aggVietnam_DoubleRice_\tRice_I_\tVietnamgee_up_load_double1.shp\tB\t7",
    "03_Rice\t04_Vietnam\tII\t2001\t2018\tTripleRice_dailyVietnam_TripleRice_\tTripleRice_8aggVietnam_TripleRice_\tRice_II_\tVietnamgee_up_load_triple1.shp\tB\t8",
    "03_Rice\t14_Cambodia\tI\t2001\t2013\tSingleRice_dailyCambodia_SingleRice_\tSingleRice_8aggCambodia_SingleRice_\tRice_I_\tCambodiagee_up_load_single.shp\tB\t9",
    "03_Rice\t07_Thailand\tI\t2001\t2017\tRice_dailyThailand_Rice_\tRice_8aggThailand_Rice_\tRice_I_\tThailand_gee_up_load.shp\tB\t10",
    "03_Rice\t11_Myanmar\tI\t2001\t2016\tSingleRice_dailyMyanmar_SingleRice_\tpre\tRice_I_\tMyanmargee_up_load_single.shp\tB\t11", # 正在下载数据
    "03_Rice\t06_Bangladesh\tI\t2001\t2017\tTripleRice_dailyBangladesh_TripleRice_\tTripleRice_8aggBangladesh_TripleRice\tRice_I_\tBangladeshgee_up_load_triple2.shp\tB\t12",
    "03_Rice\t02_India\tI\t2001\t2018\tSingleRRice_dailyIndiaI_\tSingleRice_8aggIndiaI_\tRice_I_\tIndiagee_up_load_I.shp\tB\t13",
    "03_Rice\t02_India\tII\t2001\t2018\tEarlyRice_dailyIndiaII_\tEarlyRice_8aggIndiaII_\tRice_II_\tIndiagee_up_load_II.shp\tB\t14",
    "03_Rice\t05_Pakistan\tI\t2001\t2014\tRiceI_\tSingleRice_8aggPakistan_SingleRice_\tRice_I_\tPakistangee_up_load_single.shp\tB\t15",
    "03_Rice\t12_US\tI\t2001\t2018\tRice_dailyUS_I_\tUS_8aggUS_Rice_\tRice_I_\tUSgee_up_load.shp\tB\t16",
    "03_Rice\t09_Brazil\tI\t2001\t2019\tRice_dailyBrazil_I_\tRice_8aggBrazilI_\tRice_I_\tBrazilgee_up_load.shp\tB\t17" ,
    
    ###大豆44
    "04_Soybean\t07_China\tI\t2009\t2017\tSoybean_ChinaI\tsoybean_8aggChinaI_\tSoybean_I_1\tChina_Final_shp.shp\tB\t1",
    "04_Soybean\t06_India\tI\t2001\t2018\tSoybean_India_\tsoybean_8aggIndia_\tSoybean_I_\tIndiagee_up_load.shp\tB\t2",# 有错误
    "04_Soybean\t05_Canada\tI\t2007\t2019\tSoybean_Canada_\tsoybean_8aggCanada_\tSoybean_I_\tfiltered_data_ori_gee_up_load.shp\tB\t3",
    "04_Soybean\t02_US\tI\t2001\t2018\tsoybean_dailyUSI_\tSoybean_dailyUSI_\tSoybean_I_\tUS_mergeall.shp\tB\t4", # 还在下载
    "04_Soybean\t01_Brazil\tI\t2001\t2019\tSoybean_BrazilI_\tSoybean_BrazilI_\thist_\tBrazilgee_up_load_reproject.shp\tB\t5" ,
    "04_Soybean\t03_Argentina_Uruguay\tI\t2001\t2017\tSoybean_Argentina_Uruguay_\tsoybean_8aggArgentina_Uruguay_\tSoybean_I_\tgee_up_load.shp\tB\t6",

]


for line in data_lines:
    crop,countryID, region, startyear, endyear,pre_name1,pre_name2,pre_name3,shp_name,flag,number= parse_line(line)
    print(crop,countryID, region)
    startyear = int(startyear);endyear =int(endyear)
    inputpath_base = os.path.join(root_directory,'SCI','SCI9_1','02_data',crop,countryID)
    startyear = startyear;Forecastyear = endyear # 模型开始的日期
    years = range(startyear,endyear+1);country =countryID[3:]
    years_str = [str(year) for year in years]
    SelFeature_infornamtion = extract_selected_variables(inputpath_base)
    ###################读取所选因子变量#########################################

    data= pd.read_csv(os.path.join(inputpath_base, '01_data','05_buildmodel','03_modeldata',region+'_data_ori.csv'))
   # data = data[data['idJoin'] =='CAN_15']
    TimeFeatures_sel, Static_sel, regionID = SelFeature_infornamtion[SelFeature_infornamtion['regionID'] == region].iloc[0]
    feature_all = TimeFeatures_sel+Static_sel
    filtered_columns = [col for col in data.columns if any(feature in col for feature in feature_all)]

    # 针对 flag == 'B' 的额外筛选逻辑
    feature_all = [col for col in feature_all if 'Yield' not in col]
    filtered_columns = [col for col in filtered_columns if 'Yield' not in col]  # 去掉产量相关性
    filtered_columns = [col for col in filtered_columns if 'year.1' not in col]  # 去掉产量相关性
    Static_sel = [col for col in Static_sel if 'Yield' not in col]  # 去掉产量相关性

        
    MLdata_reduced = data[filtered_columns+[yield_type]]
    MLdata_reduced['year'] = data['year']
    # 读取所选周数
    inpath_dates = os.path.join(inputpath_base, '01_data','05_buildmodel', '02_extractdates','gs_three_periods.txt')
    gs_infornamtion = pd.read_csv(inpath_dates, delim_whitespace=True, header=None)
    gs_infornamtion.columns = ['start_point', 'peak', 'harvest_point', 'VI_select2','regionID']
    start_point, peak, harvest_point, VI_select2, region = gs_infornamtion[gs_infornamtion['regionID'] == region].iloc[0]
    ############################调参数据准备###########################################################
    data_all = MLdata_reduced;X_all = data_all.drop([yield_type], axis=1);
    
    y_all = data_all[yield_type];
    # 数据标准化
    scaler_X = StandardScaler().fit(X_all)
    X = scaler_X.transform(X_all)
    scaler_y = StandardScaler().fit(y_all.values.reshape(-1, 1))
    y = scaler_y.transform(y_all.values.reshape(-1, 1)).flatten()
    
    X = pd.DataFrame(data=X,columns=X_all.columns.tolist())
    if start_point>harvest_point:
        weeks_select_list= list(range(start_point, 47))+list(range(1, harvest_point+1))
    else:
        weeks_select_list= list(range(start_point,harvest_point+1))
        
    data_list = []
    for i in weeks_select_list:
        newfeatures =  X[[f'Week{i}_{feature}' for feature in TimeFeatures_sel] + Static_sel]
        data_i = X[[f'Week{i}_{feature}' for feature in TimeFeatures_sel] + Static_sel]
        data_list.append(data_i.values)

    
    # 将 data_list 中的所有数组堆叠成一个三维数组
    data_list = np.array(data_list)
    X_all_lstm = np.transpose(data_list, (1, 0, 2)) # 将数组重组为samples， time steps, features


    ###########################制作background数据########################3
    data_background = data[filtered_columns+['idJoin']].groupby('idJoin').mean() # 取均值
    scaler_X_background = scaler_X.transform(data_background)
    
    X_background= pd.DataFrame(data=scaler_X_background,columns=X_all.columns.tolist())
    
    
    data_list_background = []
    for i in weeks_select_list:
        newfeatures =  X_background[[f'Week{i}_{feature}' for feature in TimeFeatures_sel] + Static_sel]
        data_i = X_background[[f'Week{i}_{feature}' for feature in TimeFeatures_sel] + Static_sel]
        data_list_background.append(data_i.values)
    
    
    # 将 data_list 中的所有数组堆叠成一个三维数组
    data_list_background = np.array(data_list_background)
    X_all_background = np.transpose(data_list_background, (1, 0, 2)) # 将数组重组为samples， time steps, features
    ###########################制作background数据########################
    
    ###########################模型导入###########################
    outpathmodel = os.path.join(inputpath_base,'04_model',region, 'LSTM')
    model_path = os.path.join(outpathmodel,region+'my_lstm_model.keras')
    model = load_model(model_path)
    ###########################Shap揭示LSTM模型###########################
    # ✅ 手动使用 GradientExplainer（关键点）
    explainer = shap.GradientExplainer(model, X_all_background)
    # ✅ 正确调用 shap_values
    shap_values = explainer.shap_values(X_all_lstm)
    shap_values = shap_values.squeeze(-1)# 压缩成三维数组，最后一纬度为1
    shap_valuesnew = shap_values.reshape(shap_values.shape[0], -1) # 重塑成二维数组
    
    
    # 按照三维生成列名，静态变量每周都会有个
    newfeatures = []
    for i in weeks_select_list:
        features_this_week = [f'Week{i}_{feature}' for feature in TimeFeatures_sel] + Static_sel
        newfeatures.extend(features_this_week)
    
    # 只生成动态变量名称
    newfeaturesday = []
    for i in weeks_select_list:
        features_this_week = [f'Week{i}_{feature}' for feature in TimeFeatures_sel]
        newfeaturesday.extend(features_this_week)
    
    # 将其储存为2维dataframe
    Shape_AllYear = pd.DataFrame(shap_valuesnew,columns=newfeatures)
    X_all_scale1 = pd.DataFrame(X_all_lstm.reshape(X_all_lstm.shape[0], -1),columns=newfeatures)
    Shape_AllYear['idJoin']=data['idJoin']
    X_all_scale1['idJoin']=data['idJoin']
    
    output_shapefile = os.path.join(inputpath_base, '03_results','SHAP',region)
    os.makedirs(output_shapefile, exist_ok=True)
    Shape_AllYear.to_csv(os.path.join(output_shapefile, f'{country}_{region}_shap_values.csv'), index=False)
    X_all_scale1.to_csv(os.path.join(output_shapefile, f'{country}_{region}_X_all_scale1.csv'), index=False)    
    
    
    